##                                           NLP Applications Assignment 1

##                                                       Group 80

##                                               Question Answering System

In [18]:

### Group Members Name with Student ID:#
from IPython.display import display, HTML
html_content = """

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

    <style>
        .table {
            display: table;
            width: 70%;
            border-collapse: collapse;
        }
        .table-row {
            display: table-row;
        }
        .table-cell {
            display: table-cell;
            border: 1px solid #dddddd;
            padding: 8px;
        }
        .header {
            background-color: #fffff;
            font-weight: bold;
            font-size: 100%;
        }
    </style>
</head>
<body>
 <h3>Group Members Name with Student ID:</h3>

<div class="table" style="float: left">

    <div class="table-row header">
        <div class="table-cell">BITS ID</div>
        <div class="table-cell">Name</div>
        <div class="table-cell">BITS ID</div>
        <div class="table-cell">Email ID</div>
        <div class="table-cell">Contribution</div>
    </div>
    <div class="table-row">
        <div class="table-cell">1.</div>
        <div class="table-cell">GANESHKUMAR KARUPPAIAH</div>
        <div class="table-cell">2023aa05600</div>
        <div class="table-cell">2023aa05600@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
    <div class="table-row">
    <div class="table-cell">2.</div>
        <div class="table-cell">KOLLURI VENKATESWARA SWAROOP</div>
        <div class="table-cell">2023aa05945</div>
        <div class="table-cell">2023aa05945@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
    <div class="table-row">
    <div class="table-cell">3.</div>
        <div class="table-cell">RAJESH J</div>
        <div class="table-cell">2023aa05859</div>
        <div class="table-cell">2023aa05859@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
    <div class="table-row">
    <div class="table-cell">4.</div>
        <div class="table-cell">VIJAYA LAKSHMI R</div>
        <div class="table-cell">2023aa05341</div>
        <div class="table-cell">2023aa05341@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
    <div class="table-cell">5.</div>
        <div class="table-cell">BHALANI AMIT KISHORKUMAR</div>
        <div class="table-cell">2023ab05169</div>
        <div class="table-cell">2023ab05169@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">0%</div>
    </div>
</div>
</html>
"""
display(HTML(html_content))

## Objective:

Develop a Question Answering system and apply it to Reading comprehension problem.

## Part-A:                                                                                                                                                                  [10 Marks]
Dataset for your training and validation: 
https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.jsonLinks to an external site.

1.Import Dataset and extract features and labels.
2.Use sentence embedding techniques (sentence2vec, doc2vec) to vectorize features and labels.
3.Fine tune BERT model with this dataset.
4.Your model should take question and context as input and provide answers.
5.Print answers for 5 context and questions.


In [1]:
import json
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import requests

# Load the dataset
def load_squad_dataset(file_path):
    with open(file_path, 'r') as file:
        squad_data = json.load(file)
    return squad_data

# Extract questions, contexts, and answers
def extract_features(squad_data):
    contexts, questions, answers = [], [], []

    for topic in tqdm(squad_data['data'], desc="Processing Data"):
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if not qa['is_impossible']:  # Include only answerable questions
                    question = qa['question']
                    answer = qa['answers'][0]  # Take the first answer
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    
    return pd.DataFrame({
        "context": contexts,
        "question": questions,
        "answer_text": [ans['text'] for ans in answers],
        "answer_start": [ans['answer_start'] for ans in answers]
    })

# File path to the SQuAD dataset
dataset_path = "train-v2.0.json"

# Check if the file exists
if not Path(dataset_path).is_file():
    response = requests.get("https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")
    if response.status_code == 200:
        with open(dataset_path, 'wb') as file:
            file.write(response.content)
    else:
        raise FileNotFoundError(f"Unable to download the file from the URL")

squad_data = load_squad_dataset(dataset_path)

# Extract features
df = extract_features(squad_data)

# Display a sample
print(f"Dataset Loaded: {len(df)} examples")
df.sample(5)


Processing Data: 100%|██████████| 442/442 [00:00<00:00, 15118.80it/s]

Dataset Loaded: 86821 examples


,context,question,answer_text,answer_start
79705,According to both indigenous American and Euro...,How big was the population of the Aztec cit of...,"200,000",317
88,The group changed their name to Destiny's Chil...,"Who did Beyoncé sing a duet with for ""The Best...",Marc Nelson,1212
69460,"On June 17, 2015, 21-year-old Dylann Roof ente...",What was the name of 21 year old that killed n...,Dylann Roof,30
9013,"On October 7, 2003, the recall election result...",Who was the first governor of California who h...,John G. Downey,694
41577,"On 30 October 2009, May wrote a fanclub letter...","In 2009, what was the first name of the former...",Roger,412


In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # A lightweight, fast embedding model

# Encode contexts and questions
print("Encoding contexts and questions...")
context_embeddings = model.encode(df['context'].tolist(), show_progress_bar=True)
question_embeddings = model.encode(df['question'].tolist(), show_progress_bar=True)

# Combine embeddings
print("Combining context and question embeddings...")
combined_embeddings = np.hstack((context_embeddings, question_embeddings))

# Save for further use
np.save('context_embeddings.npy', context_embeddings)
np.save('question_embeddings.npy', question_embeddings)
np.save('combined_embeddings.npy', combined_embeddings)

print("Embeddings created and saved successfully!")


Encoding contexts and questions...


Batches:   0%|          | 0/2714 [00:00<?, ?it/s]

Batches:   0%|          | 0/2714 [00:00<?, ?it/s]

Combining context and question embeddings...
Embeddings created and saved successfully!


In [7]:
# from transformers import BertForQuestionAnswering, BertTokenizer, Trainer, TrainingArguments
# from torch.utils.data import Dataset
# from sklearn.model_selection import train_test_split

# # Define Dataset class
# class SquadDataset(Dataset):
#     def __init__(self, dataframe, tokenizer, max_len=512):
#         self.dataframe = dataframe
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, index):
#         row = self.dataframe.iloc[index]
#         inputs = self.tokenizer(
#             row['question'],
#             row['context'],
#             truncation=True,
#             padding="max_length",
#             max_length=self.max_len,
#             return_tensors="pt"
#         )
#         start_position = row['answer_start']
#         end_position = start_position + len(row['answer_text'])

#         inputs['start_positions'] = start_position
#         inputs['end_positions'] = end_position
#         return {key: val.squeeze(0) for key, val in inputs.items()}

# # Load tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# # Prepare the dataset
# train_dataset = SquadDataset(df, tokenizer)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="bert-qa",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=3e-5,
#     per_device_train_batch_size=4,  # Reduced batch size
#     gradient_accumulation_steps=4,  # Simulates larger batch size
#     num_train_epochs=1,  # Fewer epochs for testing
#     weight_decay=0.01,
#     logging_dir='logs',
#     logging_steps=100,
#     # fp16=True,  # Mixed precision training
#     # dataloader_num_workers=4,  # Optimize data loading
# )



# train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
# val_dataset = SquadDataset(val_df, tokenizer)

# # If using validation
# val_dataset = SquadDataset(val_df, tokenizer)  # Optional validation set
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=SquadDataset(train_df, tokenizer),
#     eval_dataset=val_dataset  # Include this line only if you use validation
# )


# # Fine-tune the model
# print("Fine-tuning the BERT model...")
# trainer.train()

# # Save the model
# model.save_pretrained("fine_tuned_bert")
# print("save_pretrained the BERT model...")
# tokenizer.save_pretrained("fine_tuned_bert")
# print("Fine-tuned BERT model saved successfully!")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuning the BERT model...


  0%|          | 0/4883 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'loss': 6.3674, 'grad_norm': 18.53854751586914, 'learning_rate': 2.9385623592054066e-05, 'epoch': 0.02}


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'loss': 6.2989, 'grad_norm': 12.702855110168457, 'learning_rate': 2.8771247184108132e-05, 'epoch': 0.04}


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


KeyboardInterrupt: 

In [16]:
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

# Define Dataset class
class SquadDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):  # Reduced max_len
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        inputs = self.tokenizer(
            row['question'],
            row['context'],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        start_position = row['answer_start']
        end_position = start_position + len(row['answer_text'])

        inputs['start_positions'] = start_position
        inputs['end_positions'] = end_position
        return {key: val.squeeze(0) for key, val in inputs.items()}

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

# Prepare a smaller dataset
df_small = df.sample(100)  # Use a smaller subset of the dataset
train_df, val_df = train_test_split(df_small, test_size=0.1, random_state=42)

# Define training arguments
training_args = TrainingArguments(
    output_dir="distilbert-qa",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,  # No gradient accumulation
    num_train_epochs=1,  # Fewer epochs
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,  # More frequent logging
    # fp16=True,  # Mixed precision training
    # dataloader_num_workers=4,  # Optimize data loading
)

# Prepare the dataset
train_dataset = SquadDataset(train_df, tokenizer)
val_dataset = SquadDataset(val_df, tokenizer)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Fine-tune the model
print("Fine-tuning the DistilBERT model...")
trainer.train()

# Save the model
model.save_pretrained("fine_tuned_distilbert")
tokenizer.save_pretrained("fine_tuned_distilbert")
print("Fine-tuned DistilBERT model saved successfully!")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuning the DistilBERT model...


  0%|          | 0/23 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

{'loss': 4.8863, 'grad_norm': 0.0, 'learning_rate': 1.6956521739130433e-05, 'epoch': 0.43}


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

{'loss': 3.5341, 'grad_norm': 6.143878936767578, 'learning_rate': 3.913043478260869e-06, 'epoch': 0.87}


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 1.089, 'eval_samples_per_second': 9.183, 'eval_steps_per_second': 1.837, 'epoch': 1.0}
{'train_runtime': 41.2877, 'train_samples_per_second': 2.18, 'train_steps_per_second': 0.557, 'train_loss': 3.9373015320819356, 'epoch': 1.0}
Fine-tuned DistilBERT model saved successfully!


In [20]:
def answer_question(question, context):
    inputs = tokenizer(
        question, 
        context, 
        return_tensors="pt", 
        truncation=True, 
        max_length=512
    )

    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the most likely start and end of the answer span
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1

    # Convert tokens to string
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end])
    )
    return answer


In [23]:
import requests
from bs4 import BeautifulSoup

def get_wikipedia_content(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the content text
    content = soup.find('div', {'class': 'mw-parser-output'})
    
    # Extract text from the content
    text = ''
    for paragraph in content.find_all('p'):
        text += paragraph.get_text()
    
    return text

# URL of the Wikipedia page
url = 'https://en.wikipedia.org/wiki/Statistics'

# Get the content
content = get_wikipedia_content(url)
print(content)

Statistics (from German: Statistik, orig. "description of a state, a country"[1]) is the discipline that concerns the collection, organization, analysis, interpretation, and presentation of data.[2] In applying statistics to a scientific, industrial, or social problem, it is conventional to begin with a statistical population or a statistical model to be studied. Populations can be diverse groups of people or objects such as "all people living in a country" or "every atom composing a crystal". Statistics deals with every aspect of data, including the planning of data collection in terms of the design of surveys and experiments.[3]
When census data cannot be collected, statisticians collect data by developing specific experiment designs and survey samples. Representative sampling assures that inferences and conclusions can reasonably extend from the sample to the population as a whole. An experimental study involves taking measurements of the system under study, manipulating the system,

In [ ]:
# Example context
import torch

context = """
Statistics is the discipline that concerns the collection, organization, analysis, 
interpretation, and presentation of data. In applying statistics to a scientific, 
industrial, or societal problem, it is conventional to begin with a statistical 
population or a statistical model to be studied.
"""

questions = [
    "What is statistics?",
    "Where can descriptive statistics be used?",
    "How to draw meaningful conclusions?",
    "What is a p-value in statistics?", 
    "Define standard deviation.", 
    "Explain the concept of variance.", 
    "What is a normal distribution?"
]

# Generate answers
for question in questions:
    answer = answer_question(question, content)
    print(f"Q: {question}")
    print(f"A: {answer}")
    print("-" * 30)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Q: What is statistics?
A: [CLS] what is statistics ? [SEP] statistics ( from german : statistik , orig . " description of a state , a country " [ 1 ] ) is the discipline that concerns the collection , organization , analysis , interpretation , and presentation of data . [ 2 ] in applying statistics to a scientific , industrial , or social problem , it is conventional to begin with a statistical population or a statistical model to be studied . populations can be diverse groups of people or objects such as " all people living in a country " or " every atom composing a crystal " . statistics deals with every aspect of data , including the planning of data collection in terms of the design of surveys and experiments . [ 3 ] when census data cannot be collected , statisticians collect data by developing specific experiment designs and survey samples . representative sampling assures that inferences and conclusions can reasonably extend from the sample to the population as a whole . an expe

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Q: Where can descriptive statistics be used?
A: [CLS] where can descriptive statistics be used ? [SEP] statistics ( from german : statistik , orig . " description of a state , a country " [ 1 ] ) is the discipline that concerns the collection , organization , analysis , interpretation , and presentation of data . [ 2 ] in applying statistics to a scientific , industrial , or social problem , it is conventional to begin with a statistical population or a statistical model to be studied . populations can be diverse groups of people or objects such as " all people living in a country " or " every atom composing a crystal " . statistics deals with every aspect of data , including the planning of data collection in terms of the design of surveys and experiments . [ 3 ] when census data cannot be collected , statisticians collect data by developing specific experiment designs and survey samples . representative sampling assures that inferences and conclusions can reasonably extend from the s

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Q: Define standard deviation.
A: [CLS] define standard deviation . [SEP] statistics ( from german : statistik , orig . " description of a state , a country " [ 1 ] ) is the discipline that concerns the collection , organization , analysis , interpretation , and presentation of data . [ 2 ] in applying statistics to a scientific , industrial , or social problem , it is conventional to begin with a statistical population or a statistical model to be studied . populations can be diverse groups of people or objects such as " all people living in a country " or " every atom composing a crystal " . statistics deals with every aspect of data , including the planning of data collection in terms of the design of surveys and experiments . [ 3 ] when census data cannot be collected , statisticians collect data by developing specific experiment designs and survey samples . representative sampling assures that inferences and conclusions can reasonably extend from the sample to the population as a w

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Q: Explain the concept of variance.
A: deviation , and inferential statistics , which draw conclusions from data that are subject to random variation ( e . g . , observational errors , sampling variation ) . [ 4 ] descriptive statistics are most often concerned with two sets of properties of a distribution ( sample or population ) : central tendency ( or location ) seeks to characterize the distribution ' s central or typical value , while dispersion ( or variability ) characterizes the extent
------------------------------


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Q: What is a normal distribution?
A: deviation , and inferential statistics , which draw conclusions from data that are subject to random variation ( e . g . , observational errors , sampling variation ) . [ 4 ] descriptive statistics are most often concerned with two sets of properties of a distribution ( sample or population ) : central tendency ( or location ) seeks to characterize the distribution ' s central or typical value , while dispersion ( or variability ) characterizes the extent
------------------------------


Question: What is statistics?
Answer: [CLS] what is statistics ? [SEP] statistics ( from german : statistik , orig . " description of a state , a country " [ 1 ] ) is the discipline that concerns the collection , organization , analysis , interpretation , and presentation of data . [ 2 ] in applying statistics to a scientific , industrial , or social problem , it is conventional to begin with a statistical population or a statistical model to be studied . populations can be diverse groups of people or objects such as " all people living in a country " or " every atom composing a crystal " . statistics deals with every aspect of data , including the planning of data collection in terms of the design of surveys and experiments . [ 3 ] when census data cannot be collected , statisticians collect data by developing specific experiment designs and survey samples . representative sampling assures that inferences and conclusions can reasonably extend from the sample to the population as a who